# Imports

In [1]:
import sqlite3

from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker

# Introduction

The purpose of this Notebook is to learn SQLAlchemy, especially the ORM part, on a simple dataset involving authors and books.

> **NOTE**
>
> For simplicity purpose, here is assumed that a book as only one author, meaning the book to author relationship is **many to one**.
>
> One case with **many to many** relationships should be studied later.

# Defining the Models

In [2]:
# Foundational step when using the ORM
Base = declarative_base()

/tmp/ipykernel_17992/40742089.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [3]:
class Author(Base):
    __tablename__ = 'Authors'
    
    AuthorID = Column(Integer, primary_key=True, autoincrement=True)
    Name = Column(String, nullable=False)
    
    # Relationship to link to the books (an author may have many books)
    books = relationship("Book", back_populates="author")

In [4]:
class Book(Base):
    __tablename__ = 'Books'
    
    BookID = Column(Integer, primary_key=True, autoincrement=True)
    Title = Column(String, nullable=False)
    AuthorID = Column(Integer, ForeignKey('Authors.AuthorID'))  # Note this is the __tablename__ and note the class' name
    
    # Relationship to link to the author
    author = relationship("Author", back_populates="books")

# Connecting and Inserting Data

👉 We will use an in-memory SQLite instance.

This essentially means the DB will run on RAM and, hence, there won't be any persistence of data, but this isn't a problem here as we're only playing with toy

> **NOTE**
>
> It's worth to note that:
> - the data will be erased **when the connection is closed** and not **when the session is closed**.
> - by default, SQLAlchemy **isn't on auto-commit mode** and **it's best practise to commit your changes**.


In [5]:
# Create an engine and bind the metadata of the Base class to this engine
engine = create_engine('sqlite:///:memory:', echo=True)  # Using an in-memory SQLite database
Base.metadata.create_all(engine)

# Create a sessionmaker bound to the engine
Session = sessionmaker(bind=engine)

2024-05-12 12:57:30,284 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-12 12:57:30,285 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Authors")
2024-05-12 12:57:30,286 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-12 12:57:30,287 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Authors")
2024-05-12 12:57:30,288 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-12 12:57:30,289 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Books")
2024-05-12 12:57:30,290 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-12 12:57:30,291 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Books")
2024-05-12 12:57:30,292 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-05-12 12:57:30,293 INFO sqlalchemy.engine.Engine 
CREATE TABLE "Authors" (
	"AuthorID" INTEGER NOT NULL, 
	"Name" VARCHAR NOT NULL, 
	PRIMARY KEY ("AuthorID")
)


2024-05-12 12:57:30,294 INFO sqlalchemy.engine.Engine [no key 0.00059s] ()
2024-05-12 12:57:30,296 INFO sqlalchemy.engine.Engine 
CREATE TABLE "B

In [6]:
# Authors and Books data
authors = [
    Author(Name="Alice Munro"),
    Author(Name="Chimamanda Ngozi Adichie"),
    Author(Name="Gabriel García Márquez"),
    Author(Name="Haruki Murakami"),
    Author(Name="J.K. Rowling")
]

books = [
    Book(Title="Runaway", author=authors[0]),
    Book(Title="Half of a Yellow Sun", author=authors[1]),
    Book(Title="Americanah", author=authors[1]),
    Book(Title="One Hundred Years of Solitude", author=authors[2]),
    Book(Title="Love in the Time of Cholera", author=authors[2]),
    Book(Title="Norwegian Wood", author=authors[3]),
    Book(Title="Kafka on the Shore", author=authors[3]),
    Book(Title="Harry Potter and the Sorcerer's Stone", author=authors[4]),
    Book(Title="Harry Potter and the Chamber of Secrets", author=authors[4])
]

In [7]:
# Using a context manager to handle the session
with Session() as session:
    session.add_all(authors + books)
    session.commit()

2024-05-12 12:57:30,322 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-12 12:57:30,326 INFO sqlalchemy.engine.Engine INSERT INTO "Authors" ("Name") VALUES (?) RETURNING "AuthorID"
2024-05-12 12:57:30,326 INFO sqlalchemy.engine.Engine [generated in 0.00009s (insertmanyvalues) 1/5 (ordered; batch not supported)] ('Alice Munro',)
2024-05-12 12:57:30,328 INFO sqlalchemy.engine.Engine INSERT INTO "Authors" ("Name") VALUES (?) RETURNING "AuthorID"
2024-05-12 12:57:30,329 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/5 (ordered; batch not supported)] ('Chimamanda Ngozi Adichie',)
2024-05-12 12:57:30,330 INFO sqlalchemy.engine.Engine INSERT INTO "Authors" ("Name") VALUES (?) RETURNING "AuthorID"
2024-05-12 12:57:30,332 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/5 (ordered; batch not supported)] ('Gabriel García Márquez',)
2024-05-12 12:57:30,333 INFO sqlalchemy.engine.Engine INSERT INTO "Authors" ("Name") VALUES (?) RETURNING "AuthorID"
2024-05-12 12:57:30,333 INFO sqlalchem

In [8]:
# To verify insertion, print authors and their books
with Session() as session:
    for author in session.query(Author).all():
        print(f"Author: {author.AuthorID} - {author.Name}")
        for book in author.books:
            print(f" - Book: {book.Title}")
        

2024-05-12 12:57:30,368 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-12 12:57:30,372 INFO sqlalchemy.engine.Engine SELECT "Authors"."AuthorID" AS "Authors_AuthorID", "Authors"."Name" AS "Authors_Name" 
FROM "Authors"
2024-05-12 12:57:30,373 INFO sqlalchemy.engine.Engine [generated in 0.00089s] ()
Author: 1 - Alice Munro
2024-05-12 12:57:30,378 INFO sqlalchemy.engine.Engine SELECT "Books"."BookID" AS "Books_BookID", "Books"."Title" AS "Books_Title", "Books"."AuthorID" AS "Books_AuthorID" 
FROM "Books" 
WHERE ? = "Books"."AuthorID"
2024-05-12 12:57:30,379 INFO sqlalchemy.engine.Engine [generated in 0.00156s] (1,)
 - Book: Runaway
Author: 2 - Chimamanda Ngozi Adichie
2024-05-12 12:57:30,382 INFO sqlalchemy.engine.Engine SELECT "Books"."BookID" AS "Books_BookID", "Books"."Title" AS "Books_Title", "Books"."AuthorID" AS "Books_AuthorID" 
FROM "Books" 
WHERE ? = "Books"."AuthorID"
2024-05-12 12:57:30,383 INFO sqlalchemy.engine.Engine [cached since 0.004962s ago] (2,)
 - Book: Half o

# Workout On Retrieving Data

## Retrieving All Authors

In [9]:
with Session() as session:
    authors = session.query(Author).all()
    for author in authors:
        print(author.Name)

2024-05-12 12:57:30,406 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-12 12:57:30,408 INFO sqlalchemy.engine.Engine SELECT "Authors"."AuthorID" AS "Authors_AuthorID", "Authors"."Name" AS "Authors_Name" 
FROM "Authors"
2024-05-12 12:57:30,409 INFO sqlalchemy.engine.Engine [cached since 0.03692s ago] ()
Alice Munro
Chimamanda Ngozi Adichie
Gabriel García Márquez
Haruki Murakami
J.K. Rowling
2024-05-12 12:57:30,412 INFO sqlalchemy.engine.Engine ROLLBACK


## Retrieving All Books

In [10]:
with Session() as session:
    books = session.query(Book).all()
    for book in books:
        print(book.Title)

2024-05-12 12:57:30,421 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-12 12:57:30,424 INFO sqlalchemy.engine.Engine SELECT "Books"."BookID" AS "Books_BookID", "Books"."Title" AS "Books_Title", "Books"."AuthorID" AS "Books_AuthorID" 
FROM "Books"
2024-05-12 12:57:30,425 INFO sqlalchemy.engine.Engine [generated in 0.00106s] ()
Runaway
Half of a Yellow Sun
Americanah
One Hundred Years of Solitude
Love in the Time of Cholera
Norwegian Wood
Kafka on the Shore
Harry Potter and the Sorcerer's Stone
Harry Potter and the Chamber of Secrets
2024-05-12 12:57:30,427 INFO sqlalchemy.engine.Engine ROLLBACK


## Retrieve Books by a Specific Author

In [11]:
with Session() as session:
    books = (session
             .query(Book)
             .join(Author)
             .filter(Author.Name == 'J.K. Rowling')
             .all())
    for book in books:
        print(f"{book.Title} by {book.author.Name}")

2024-05-12 12:57:30,436 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-12 12:57:30,439 INFO sqlalchemy.engine.Engine SELECT "Books"."BookID" AS "Books_BookID", "Books"."Title" AS "Books_Title", "Books"."AuthorID" AS "Books_AuthorID" 
FROM "Books" JOIN "Authors" ON "Authors"."AuthorID" = "Books"."AuthorID" 
WHERE "Authors"."Name" = ?
2024-05-12 12:57:30,440 INFO sqlalchemy.engine.Engine [generated in 0.00076s] ('J.K. Rowling',)
2024-05-12 12:57:30,442 INFO sqlalchemy.engine.Engine SELECT "Authors"."AuthorID" AS "Authors_AuthorID", "Authors"."Name" AS "Authors_Name" 
FROM "Authors" 
WHERE "Authors"."AuthorID" = ?
2024-05-12 12:57:30,442 INFO sqlalchemy.engine.Engine [generated in 0.00068s] (5,)
Harry Potter and the Sorcerer's Stone by J.K. Rowling
Harry Potter and the Chamber of Secrets by J.K. Rowling
2024-05-12 12:57:30,444 INFO sqlalchemy.engine.Engine ROLLBACK


In [12]:
# A more complex one, with the `contains` method
with Session() as session:
    books = (session
             .query(Book)
             .join(Author)
             .filter(Author.Name.contains("Haruki"))
             .all())
    for book in books:
        print(f"{book.Title} by {book.author.Name}")

2024-05-12 12:57:30,454 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-12 12:57:30,456 INFO sqlalchemy.engine.Engine SELECT "Books"."BookID" AS "Books_BookID", "Books"."Title" AS "Books_Title", "Books"."AuthorID" AS "Books_AuthorID" 
FROM "Books" JOIN "Authors" ON "Authors"."AuthorID" = "Books"."AuthorID" 
WHERE ("Authors"."Name" LIKE '%' || ? || '%')
2024-05-12 12:57:30,458 INFO sqlalchemy.engine.Engine [generated in 0.00144s] ('Haruki',)
2024-05-12 12:57:30,460 INFO sqlalchemy.engine.Engine SELECT "Authors"."AuthorID" AS "Authors_AuthorID", "Authors"."Name" AS "Authors_Name" 
FROM "Authors" 
WHERE "Authors"."AuthorID" = ?
2024-05-12 12:57:30,460 INFO sqlalchemy.engine.Engine [cached since 0.01872s ago] (4,)
Norwegian Wood by Haruki Murakami
Kafka on the Shore by Haruki Murakami
2024-05-12 12:57:30,462 INFO sqlalchemy.engine.Engine ROLLBACK


## Query Books With Multiple Conditions

In [13]:
with Session() as session:
    books = (session
             .query(Book)
             .join(Author)
             .filter(Book.Title.contains("The"), Author.Name.contains("Alice"))
             .all())
    for book in books:
        print(f"{book.Title} by {book.author.Name}")

2024-05-12 12:57:30,475 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-12 12:57:30,478 INFO sqlalchemy.engine.Engine SELECT "Books"."BookID" AS "Books_BookID", "Books"."Title" AS "Books_Title", "Books"."AuthorID" AS "Books_AuthorID" 
FROM "Books" JOIN "Authors" ON "Authors"."AuthorID" = "Books"."AuthorID" 
WHERE ("Books"."Title" LIKE '%' || ? || '%') AND ("Authors"."Name" LIKE '%' || ? || '%')
2024-05-12 12:57:30,478 INFO sqlalchemy.engine.Engine [generated in 0.00081s] ('The', 'Alice')
2024-05-12 12:57:30,480 INFO sqlalchemy.engine.Engine ROLLBACK


## Find Authors Without Any Book

In [14]:
with Session() as session:
    authors_without_books = (session
                             .query(Author)
                             .outerjoin(Book, Author.books)
                             .filter(Book.BookID == None)
                             .all())
    for author in authors_without_books:
        print(author.Name)

2024-05-12 12:57:30,490 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-05-12 12:57:30,493 INFO sqlalchemy.engine.Engine SELECT "Authors"."AuthorID" AS "Authors_AuthorID", "Authors"."Name" AS "Authors_Name" 
FROM "Authors" LEFT OUTER JOIN "Books" ON "Authors"."AuthorID" = "Books"."AuthorID" 
WHERE "Books"."BookID" IS NULL
2024-05-12 12:57:30,494 INFO sqlalchemy.engine.Engine [generated in 0.00136s] ()
2024-05-12 12:57:30,496 INFO sqlalchemy.engine.Engine ROLLBACK


# Close the Connection

In [15]:
engine.dispose()